# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['body_text'])
x_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(x_tfidf.toarray())], axis=1)

#convert column names
tfidf_column_numbers = x_tfidf_feat.columns
tfidf_column_strings = [str(x) for x in tfidf_column_numbers]
x_tfidf_feat.columns = tfidf_column_strings

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
x_count = count_vect.fit_transform(data['body_text'])
x_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(x_count.toarray())], axis=1)


#convert column names
count_column_numbers = x_count_feat.columns
count_column_strings = [str(x) for x in count_column_numbers]
x_count_feat.columns = count_column_strings

x_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
gb = GradientBoostingClassifier()

param = {
    
    'n_estimators':[100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
    
}

#cross-validation is to evaluate the model over the entire data-set and works much better

#n_jobs here is to test the models parallelly, not to train decision trees inside the model paralelly.
gs = GridSearchCV(gb, param, cv=5, n_jobs = -1)
cv_fit = gs.fit(x_tfidf_feat, data['label'])

pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,290.074384,29.498439,0.165392,0.010070,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964093,0.976661,0.973046,0.969452,0.968553,0.970361,0.004249,1
3,324.165949,3.931517,0.263974,0.053763,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.977558,0.969452,0.966757,0.966757,0.969282,0.004308,2
0,144.888272,3.601930,0.284790,0.014472,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964991,0.977558,0.969452,0.965858,0.967655,0.969103,0.004498,3
4,266.701037,12.311886,0.292372,0.074013,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964093,0.974865,0.969452,0.966757,0.969452,0.968924,0.003575,4
2,210.033614,3.884416,0.264063,0.010442,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.978456,0.967655,0.965858,0.966757,0.968743,0.004937,5


In [4]:
gb = GradientBoostingClassifier()

param = {
    
    'n_estimators':[100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
    
}

#cross-validation is to evaluate the model over the entire data-set and works much better

#n_jobs here is to test the models parallelly, not to train decision trees inside the model paralelly.
gs = GridSearchCV(gb, param, cv=5, n_jobs = -1)
cv_fit = gs.fit(x_count_feat, data['label'])

pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,249.365080,29.110158,0.153753,0.011608,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.978456,0.969452,0.963163,0.972147,0.969283,0.005777,1
3,298.394044,3.520652,0.212001,0.022215,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.962298,0.980251,0.966757,0.966757,0.970350,0.969283,0.006050,2
1,208.883443,3.020406,0.298142,0.036985,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965889,0.978456,0.970350,0.962264,0.966757,0.968743,0.005495,3
2,201.319682,3.207552,0.251385,0.025102,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.974865,0.967655,0.963163,0.970350,0.968384,0.003997,4
0,133.847002,3.559391,0.248492,0.005169,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.970350,0.962264,0.966757,0.968384,0.005724,5
